In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 03: Agent-as-Tool Collaboration Pattern

This notebook demonstrates the **Agent-as-Tool** collaboration paradigm, where one agent uses another specialized agent as a tool. The financial assistant from Lecture 01 is enhanced with a `stock_report` tool that internally delegates to the Lecture 02 stock analysis agent.

## How Agent-as-Tool Works

This extends the ReAct pattern from Lecture 01 by adding multiple tool options. The agent now chooses between different tools based on the type of information needed:

```mermaid
graph TD
    Start([👤 User Question 👤]) --> Reason[🤖 Agent: Analyze & Reason<br/>- Understand question<br/>- Plan approach<br/>- Decide on tools]
    Reason --> Decision{Need<br/>Tools?}
    
    Decision -->|No| Response[📝 Generate<br/>Final Response]
    Decision -->|Yes| ToolChoice{Which<br/>Tool?}
    
    ToolChoice -->|General Info| WebSearch[🔧 Web Search&nbsp;<br/>Tool]
    ToolChoice -->|Stock Analysis| StockReport[🔧 Stock Report Tool&nbsp;<br/>Agent-as-Tool]
    
    WebSearch --> Results[📊 Receive<br/>Tool Results]
    StockReport -->|Delegates to<br/>Stock Analyzer Agent| StockResults[📊 Receive<br/>StockReport Object]
    StockResults --> Results
    
    Results --> React[🤖 Agent: React<br/>- Process results<br/>- Integrate information<br/>- Decide next step]
    
    React --> Decision
    
    Response --> End([✅ Return Answer&nbsp;<br/>to User])
    
    style Start fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Reason fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Decision fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style ToolChoice fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style WebSearch fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style StockReport fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style Results fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style StockResults fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style React fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Response fill:#e0f2f1,stroke:#004d40,stroke-width:2px
    style End fill:#e1f5ff,stroke:#01579b,stroke-width:3px
```

**Key Differences from Lecture 01:**
- **Multiple Tools**: Agent now has 2 tools to choose from (web_search and stock_report)
- **Tool Selection**: Additional decision node for choosing the appropriate tool
- **Agent-as-Tool**: The stock_report tool is actually another agent (from Lecture 02) that runs independently
- **Structured Output**: Stock analysis returns typed `StockReport` objects instead of raw text

**Key Benefits:**
- **Modularity**: Specialized agents can be reused as tools by other agents
- **Separation of Concerns**: Each agent focuses on its specific domain
- **Type Safety**: Agents return structured Pydantic models ensuring data consistency
- **Reusability**: The stock analysis agent can be used standalone or as a tool


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import Tool
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging

# Initialize logging
setup_logging()

In [2]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="tool-delegation-lecture03",
    start_message="🚀 Lecture 03 - Tool Delegation Notebook Started",
)

In [3]:
# Define the StockReport model inline for self-contained presentation
from typing import List, Optional
from pydantic import BaseModel, Field
from IPython.display import Markdown, display


class StockReport(BaseModel):
    """Comprehensive stock analysis report."""

    symbol: str = Field(description="Stock symbol (e.g., AAPL, TSLA)")
    company_name: str = Field(description="Full company name")
    current_price: Optional[float] = Field(
        description="Current stock price", default=None
    )
    price_change: Optional[float] = Field(
        description="Price change from previous close", default=None
    )
    price_change_percent: Optional[float] = Field(
        description="Price change percentage", default=None
    )

    # Analysis sections
    executive_summary: str = Field(description="Brief executive summary of the stock")
    recent_news: List[str] = Field(
        description="Key recent news items affecting the stock", default_factory=list
    )
    financial_highlights: List[str] = Field(
        description="Key financial metrics and highlights", default_factory=list
    )
    market_sentiment: str = Field(description="Overall market sentiment analysis")
    risk_factors: List[str] = Field(
        description="Key risk factors to consider", default_factory=list
    )
    recommendation: str = Field(description="Investment recommendation with reasoning")

    # Metadata
    analysis_date: str = Field(description="Date when analysis was performed")
    data_sources: List[str] = Field(
        description="Sources used for the analysis", default_factory=list
    )

In [4]:
# Define helper functions and the analyze_stock agent inline
import logging
from datetime import datetime

logger = logging.getLogger(__name__)


def get_current_date() -> str:
    """Get current date in YYYY-MM-DD format."""
    return datetime.now().strftime("%Y-%m-%d")


def validate_stock_symbol(symbol: str) -> str:
    """Validate and normalize stock symbol."""
    if not symbol:
        raise ValueError("Stock symbol cannot be empty")

    # Convert to uppercase and remove whitespace
    normalized = symbol.strip().upper()

    # Basic validation - should be 1-5 characters, alphanumeric
    if not normalized.isalnum() or len(normalized) > 5:
        raise ValueError(f"Invalid stock symbol format: {symbol}")

    return normalized


# Create the stock analysis agent (this is what was in lecture02)
stock_analysis_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    output_type=StockReport,
    system_prompt=f""""
You are an expert financial analyst specializing in stock analysis and market research.

IMPORTANT CONTEXT:
- Today's date is {get_current_date()}
- Always provide information in the context of this current date

Your task is to create comprehensive stock reports by gathering and analyzing current information about publicly traded companies.

CAPABILITIES:
- You have access to a web search tool that can find real-time information about stocks, companies, and market conditions
- You can search for financial news, earnings reports, analyst opinions, and market data
- You should gather information from multiple reliable sources

ANALYSIS FRAMEWORK:
When analyzing a stock, you should:

1. **Company Overview**: Get basic company information, business model, and recent developments
2. **Financial Performance**: Look for recent earnings, revenue trends, and key financial metrics
3. **Market Position**: Research competitive landscape and market share
4. **Recent News**: Find recent news that could impact the stock price
5. **Analyst Opinions**: Look for professional analyst ratings and price targets
6. **Risk Assessment**: Identify potential risks and challenges
7. **Market Sentiment**: Gauge overall market sentiment toward the stock

SEARCH STRATEGY:
- Use multiple targeted searches to gather comprehensive information
- Search for recent news (last 3-6 months) for current relevance
- Look for both positive and negative information to provide balanced analysis
- Verify information from multiple sources when possible
- Focus on reputable financial news sources and official company communications

OUTPUT REQUIREMENTS:
- Provide a structured analysis following the StockReport format
- Include specific data points when available (prices, percentages, dates)
- Cite your sources and indicate when information is current
- Be objective and balanced in your analysis
- Clearly distinguish between facts and opinions/projections
- If certain information is not available, acknowledge this rather than speculating

Remember: Your analysis should be thorough, factual, and useful for investment decision-making.
    """.strip(),
)


async def analyze_stock(ctx: RunContext, symbol: str) -> StockReport:
    """
    Analyze a stock and generate a comprehensive report.

    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'TSLA')

    Returns:
        StockReport: Comprehensive analysis report
    """
    # Validate and normalize the stock symbol
    normalized_symbol = validate_stock_symbol(symbol)

    logger.info(f"Starting stock analysis for: {normalized_symbol}")

    # Create the analysis prompt
    prompt = f"""
Please analyze the stock {normalized_symbol} and provide a comprehensive report.

The analysis date should be: {get_current_date()}
    """.strip()

    try:
        # Run the agent analysis
        result = await stock_analysis_agent.run(prompt)

        logger.info(f"Stock analysis completed for {normalized_symbol}")
        logger.debug(f"Analysis result: {result.output}")

        return result.output

    except Exception as e:
        logger.error(f"Error analyzing stock {normalized_symbol}: {str(e)}")
        raise

In [ ]:
# Create a tool that delegates to the lecture02 stock analysis agent
stock_report_tool = Tool(
    analyze_stock,
    description="Get a detailed stock analysis report for a stock symbol (e.g., AAPL, TSLA).",
)

# Create enhanced financial assistant with both web_search and stock_report tools
enhanced_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, stock_report_tool],  # Now has 2 tools
    system_prompt=f"""
You are a knowledgeable financial assistant.

IMPORTANT CONTEXT:
- Today's date is {get_current_date()}
- Always provide information in the context of this current date

When answering questions:
1. Use the web_search tool for general financial information
2. Use the stock_report tool when users ask for detailed analysis of specific stocks
3. Combine information from both sources with your financial knowledge
4. Provide clear, actionable advice
5. Include appropriate disclaimers for investment advice

Keep your responses informative but concise.
""",
)

In [6]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await enhanced_financial_agent.run(question)
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

**Question:** What is the current price of Bitcoin?

---

As of today (2025-12-15), the current price of Bitcoin (BTC) is approximately \$88,250 to \$88,470 USD, according to the latest data from multiple financial platforms.

Please note that cryptocurrency prices are highly volatile and can change rapidly. Always check a reputable exchange for the most current price before making any trading or investment decisions.

In [7]:
# Ask for detailed stock analysis (uses stock_report tool)
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await enhanced_financial_agent.run(question)
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

2025-12-15 10:00:35 - __main__ - INFO - Starting stock analysis for: AAPL
2025-12-15 10:00:52 - __main__ - INFO - Stock analysis completed for AAPL


**Question:** Can you give me a detailed analysis of Apple stock (AAPL)?

---

Here is a detailed analysis of Apple Inc. (AAPL) as of December 15, 2025:

Current Status:
- Stock price: \$278.78 (near all-time highs)
- Market cap: \$4.11 trillion, #2 globally
- YTD total return: 16%, rebounding after early volatility

Fundamentals & Financial Highlights:
- 2025 revenue: \$416.16 billion (+6.4% YoY)
- Q4 2025 revenue: \$102.47 billion
- Gross profit margin: 47%+; Services segment topline >\$100B (high margin)
- Operating margin (LTM): ~32%; Free cash flow margin: ~23.5%
- PE ratio: ~40x (premium valuation)
- Earnings and EPS at record highs

Recent News & Developments:
- iPhone 17 cycle showing above-expected demand
- Early AI integration (Siri, iOS); Wall Street bullish on future AI monetization
- Major executive transitions, including COO and AI leadership changes
- Faces antitrust lawsuits in US/EU; regulatory risk is elevated

Market Sentiment:
- Consensus rating: Buy; most analyst targets in \$280–\$330 range, bulls as high as \$350
- Constructive sentiment driven by product strength and service growth, tempered by AI and regulatory uncertainty
- Short-term upside modest (3-7%); long-term upside depends on AI execution and regulatory outcomes

Risks:
- Antitrust/regulatory headwinds could hit high-margin Services
- Leadership turnover (COO, legal/AI heads) raises continuity/execution risk
- Geopolitical/trade risk (especially China) & macro sensitivity
- Competition, especially in AI
- Elevated valuation leaves little room for execution missteps

Recommendation:
Apple remains a high-quality core holding given its fundamentals, innovation, and brand strength. New allocations should be sized conservatively and closely monitored for developments in AI strategy and regulatory actions. Maintain or cautiously accumulate on weakness; avoid aggressive buying until more clarity in leadership and regulatory landscape emerges.

Disclaimer: This analysis is for informational purposes and should not be construed as personalized investment advice. Please consult your financial advisor before making investment decisions.

In [9]:
# Try another stock analysis
question = "Find 3 trending stocks and provide an investor report for them"
result = await enhanced_financial_agent.run(question)
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
display(Markdown(result.output.replace("$", "\\$")))

2025-12-13 10:47:51 - __main__ - INFO - Starting stock analysis for: REXR
2025-12-13 10:47:51 - __main__ - INFO - Starting stock analysis for: ELF
2025-12-13 10:47:51 - __main__ - INFO - Starting stock analysis for: STRZ
2025-12-13 10:48:01 - __main__ - INFO - Stock analysis completed for REXR
2025-12-13 10:48:04 - __main__ - INFO - Stock analysis completed for ELF
2025-12-13 10:48:09 - __main__ - INFO - Stock analysis completed for STRZ


**Question:** Find 3 trending stocks and provide an investor report for them

---

Here are investor reports on three trending stocks as of December 2025:

---

### 1. Rexford Industrial Realty (REXR)
- **Price (Dec 13, 2025):** \$40.73
- **Profile:** A real estate investment trust (REIT) specializing in Southern California industrial properties.
- **Fundamentals:** Q3 revenue \$253.2M, Q3 EPS \$0.37 (missed estimates). Net income for 9 months: \$268.9M. Dividend yield ~4.1%. Occupancy solid at 89.2%.
- **News:** CEO succession announced; insider selling by a top executive; new board appointment.
- **Market Sentiment:** Neutral to slightly bearish. Analysts rate “Hold,” citing stable cash flows but caution due to earnings miss and sector headwinds.
- **Risks:** Higher interest expenses, dilution risk, moderate occupancy decline from repositioning efforts.
- **Recommendation:** A stable, income-focused option for long-term holders. Hold for now; dividend remains attractive. Limited short-term upside due to headwinds.

---

### 2. e.l.f. Beauty (ELF)
- **Price (Dec 13, 2025):** \$77.96
- **Profile:** Fast-growing, digital-first beauty company with clean, affordable products; strong e-commerce strategy.
- **Fundamentals:** Q2 FY2026 EPS \$0.68 (beat by \$0.11); FY2025 sales up 28% to \$1.31B. Analysts’ average price target is \$127.50.
- **News:** CEO sold shares this week; strong brand partnership results; recent earnings beat.
- **Market Sentiment:** Mixed but leaning bullish. High growth and price targets, but recent pullback and insider selling spark caution.
- **Risks:** Insider selling, steep price drop (down 42% from 2024 highs), rising competition, premium valuation, consumer spending trends.
- **Recommendation:** Appears undervalued for long-term growth investors. Attractive if you tolerate volatility, but watch near-term risks.

---

### 3. Starz Entertainment (STRZ)
- **Price (Dec 13, 2025):** \$10.74 (–18.6% recent drop)
- **Profile:** Premium television and streaming provider, recently split from Lionsgate. Repositioning its business toward digital.
- **Fundamentals:** Q3 revenue \$320.9M, net loss \$52.6M; annual revenue declining, negative earnings, and high leverage.
- **News:** Major insider selling; widening quarterly losses; preparing for big franchise releases in 2026.
- **Market Sentiment:** Cautious to negative. Bearish outlook due to ongoing losses, falling revenues, and insider selling.
- **Risks:** Revenue decline, high debt, intense streaming competition, management confidence concerns.
- **Recommendation:** Hold. Speculative and only suitable for risk-tolerant investors. Avoid entry unless clear signs of turnaround appear.

---

**Disclaimer:** These summaries are for informational purposes only and do not constitute investment advice. Please do your own research or consult a qualified advisor before making investment decisions.